In [133]:
import pandas as pd
from r import *
from r2 import *
import json

data = pd.read_csv("./dataset.csv")
formule = data['Equation']


In [134]:
import random
from r import calculate_tree, fix2
import numpy as np

def mutate_value(expression):
    tokens = parse_equation1(expression)
    for i,c in enumerate(tokens):
        # print(c)
        if c not in "x*/()-+" and c != "x":
            mutateBool = [True, False, False, False, False]
            if random.choice(mutateBool):
                rand_vals = [str(random.randint(-9, -1)), str(random.randint(1, 9))]
                new_val = random.choice(rand_vals)
                tokens[i] = new_val
    return tokens


def mutate_operation(expression):
    tokens = parse_equation1(expression)
    for i,c in enumerate(tokens):
        # print(c)
        if c in "*/-+^":
            mutateBool = [True, False, False, False, False]
            if random.choice(mutateBool):
                rand_op = ["+", "-", "*","/", "**"]
                new_val = random.choice(rand_op)
                tokens[i] = new_val
    return tokens
           


In [135]:
def fitness_MSE(guess_eq, df, index):
    row = df.iloc[index]
    Xs = json.loads(row['Xs'])
    
    Ys = json.loads(row['Ys'])
    real_eq = row['Equation']

    tokens = parse_equation1(guess_eq)

    fix_tokens = fix2(tokens)
    sorted_tokens = sort_tokens(fix_tokens)
    tree = build_tree(sorted_tokens)
    fitnessVal = 0

    MSE_error = []
    for i in range(len(Xs)):
        rez = calculate_tree(tree, int(Xs[i]))
        real_val = int(Ys[i])
        sub = real_val - rez
        if (sub > 100000):
            return 16284124680
        MSE = round((sub)**2, 1)
        

        MSE_error.append(MSE)
    
    return round(sum(MSE_error)//len(MSE_error),1)

In [136]:
def fitness_primitive(guess_eq, df, index):
    row = df.iloc[index]
    Xs = json.loads(row['Xs'])
    
    Ys = json.loads(row['Ys'])
    real_eq = row['Equation']

    tokens = parse_equation1(guess_eq)

    fix_tokens = fix2(tokens)
    sorted_tokens = sort_tokens(fix_tokens)
    tree = build_tree(sorted_tokens)
    fitnessVal = 0
    for i in range(len(Xs)):
        rez = calculate_tree(tree, int(Xs[i]))
        real_val = int(Ys[i])
        sub = round(np.abs(real_val - rez), 4)
        # if sub > 1000000000:
        #     return 0
        if sub == 0:
            fitnessVal += 999
        else:
            fitnessVal += round(1/sub,5)
    return fitnessVal
        
    
def generate_expression(numTerms, x_index):
    operators = ["+", "-", "*","/", "**"]
    vals = [str(random.randint(1, 9)),str(random.randint(1, 9)), "x"]
    vals2 = str(random.randint(1, 9))
    
    num_terms = random.randint(numTerms, 2+numTerms)
    
    x_index %= numTerms
    correct = False
    while not correct:
        correct = True
        terms = []
        del terms[:]
        x = 0
        for i in range(num_terms):
            
            op = random.choice(operators)
            num = random.choice(vals)

            if x_index == i:
                if i > 0 and terms[i-1] == "**":
                    correct = False
                    break     #da ne dobimo neki**x
                else:
                    num = "x"
                    x += 1
            else:
                if i > 0 and terms[i-1] == "**":
                    
                    if num == "x":
                        correct = False
                        break
            terms.append(num)
            terms.append(op)

        terms.append(vals2)
        expression = "".join(terms)

    return expression

expression = generate_expression(3, 21)
print(expression)
# fitnesVal = fitness_primitive(expression, data, 1)
# print(fitnesVal)

x**1*1+7-6


In [137]:
# N = 100
def sort_dict(dict):
    return {k: dict[k] for k in sorted(dict, reverse=True)}

def arr_to_string(array):
    return ''.join(map(str, array))



def pick2trees(dictionary):
    e1 = random.choice(tuple(dictionary.items()))
    e2 = random.choice(tuple(dictionary.items()))

    tree_e1 = get_exp_fromArr(str(e1[1]))
    tree_e2 = get_exp_fromArr(str(e2[1]))

    # k1 = print_infix(tree_e1)
    # k2 = print_infix(tree_e2)

    return tree_e1, tree_e2


equationIndex = 10

    

def ga(N, gen, equationIndex):
    
    
    prevN = N
    best_of_gens = {}
    for i in range(gen):
        # print(i+1)
        results = {}
        N = prevN

        for i in range(N):
            expression = generate_expression(3, i)
            fitness = fitness_primitive(expression, data, equationIndex)
        
            results[fitness] = expression
    
        N = N // 2
        
        result = sort_dict(results)
        new_dict = dict(list(result.items())[:N])

        while N > 1:
            for ele in new_dict:
                mutation_choice = random.choice([0,0,0,0,0,0,1,1, 1,1,1,1, 2,3,3,3,3,3,3, 3])

                if mutation_choice == 1:
                    expression = arr_to_string(mutate_value(result[ele]))   
                    
                    fitness = round(fitness_primitive(expression, data, equationIndex),1)
                    
                    result[fitness] = expression
                if mutation_choice == 2:
                    expression = arr_to_string(mutate_operation(result[ele]))

                    fitness = round(fitness_primitive(expression, data, equationIndex), 1)
                    result[fitness] = expression

                if mutation_choice == 3:
                    tree1, tree2 = pick2trees(new_dict)
                    tree1, tree2 = crossover(tree1, tree2)

                    #ce ni x v enacbi ponavlja 
                    while(check_if_x(tree1) == False) or (check_if_x(tree2) == False):
                        tree1_str = print_infix_parenthesis(tree1)

                        tree2_str = print_infix_parenthesis(tree2)
                        # print(f"tree1: {tree1_str}, tree2: {tree2_str}")
                        tree1,tree2 = crossover(tree1,tree2)
                        

                    tree1_str = infixString(tree1)
                    tree2_str = infixString(tree2)
                    
                    fitness1 = round(fitness_primitive(tree1_str, data, equationIndex), 1)
                    fitness2 = round(fitness_primitive(tree2_str, data, equationIndex),1)


                    result[fitness1] = tree1_str
                    result[fitness2] = tree2_str

                    

            N = N // 2
            result = sort_dict(results)
            new_dict = dict(list(result.items())[:N]) 
        

        val = list(sort_dict(results))[0]
        best_of_gens[val] = results[val]
        # print("best of gen: " + results[val] + " " + str(val))

    return best_of_gens

k = ga(1000, 10, equationIndex)
k = sort_dict(k)

val = list(k)[0]

row = data.iloc[equationIndex]
Xs = json.loads(row['Xs'])

Ys = json.loads(row['Ys'])
real_eq = row['Equation']
print("---------------------")

print(real_eq)
print("---------------------")

print(k[val], val)




---------------------
(((x - 2) + 4) * 5)
---------------------
x/x+x*5+9 99900


In [144]:

for eqIndex in range(0, len(data)-80):
    
    k = ga(1000, 10, eqIndex)
    k = sort_dict(k)

    val = list(k)[0]

    row = data.iloc[eqIndex]
    Xs = json.loads(row['Xs'])

    Ys = json.loads(row['Ys'])
    real_eq = row['Equation']
    print("---------------------")
    print(f"{eqIndex} - real eq: " + real_eq)
    print("=====================")
    print(k[val], val)
    print("---------------------")


---------------------
0 - real eq: ((x ** 4) - 6)
x**4-1/4-1-5 55944.0
---------------------
---------------------
1 - real eq: (((x / 8) * 2) + 1)
x/x+x/4 99900
---------------------
---------------------
2 - real eq: (((x - 1) - 3) / 5)
8/5**x+x/5-1 77944.0
---------------------
---------------------
3 - real eq: (x * 5)
x*5**5/5**4 99900
---------------------
---------------------
4 - real eq: (x + 2)
x/x*x+2 99900
---------------------
---------------------
5 - real eq: ((x + 1) * 7)
7+x*7*1*1*1 99900
---------------------
---------------------
6 - real eq: ((x + 2) ** 9)
x*4**x**x**2 999.00006
---------------------
---------------------
7 - real eq: (x * 2)
x+x-2/4 99900
---------------------
---------------------
8 - real eq: (((x ** 8) * 5) / 3)
x**7*5-5/5*4 999.2548
---------------------
---------------------
9 - real eq: (x / 7)
x/7-7/x**x/8 99900
---------------------
---------------------
10 - real eq: (((x - 2) + 4) * 5)
x/8*x+3+7 7993.541439999998
---------------------
---